# API-Based Data Wrangling in Python – Simple Cases
## Introduction
You may recall that the ugly truth about collecting data from an API is that every
 API source can be and usually is unique.  By that I mean, while the data
 returned will almost always be in a standard file format (CSV, JSON,
 XML), the API endpoint navigation and interactions and returned data
 schema (and thus interpretation) will most certainly be unique.

 Long story short, <u>your success will depend on your ability to read,
 interpret, and implement technical information pertaining to a
 service's APIs and their returned data.</u>

So, in order to ensure we make a successful request, when we work with
 APIs it's important to consult the documentation. Technical
 documentation can seem daunting at first, but as you use documentation
 more and more you'll find it gets easier.  And, of course, some docs
 are better than others.

## Question
The Star Wars API (SWAPI) is maintained by avid Star Wars fans.  SWAPI
 is the world's first quantified and programmatically-formatted
 Star Wars dataset.  Apparently, the data returned by the API has been
 complied after countless hours of watching films and trawling through
 content online.  The API exposes endpoints capable of providing data
 about People, Films, Species, Starships, Vehicles and Planets from
 the Star Wars movies.

The API documentation should help you familiarize yourself with the
 resources available and how to consume them with HTTP requests.  Read
 through the getting started section before you dive in. Most issues
 can be resolved by reading the documentation.  Star Wars API
 Documentation (https://swapi.dev/documentation).

Notice how several of the APIs are logically 'linked' - ie., related.  This concept is
best seen by looking at the attributes returned for the films endpoint
 (https://swapi.dev/api/films/).

The films API returns JSON data.  Within that data is an array titled characters.  The
 characters attribute is an array of other SWAPI endpoints that that can be used to
 return the people endpoint data for all of the characters in the film.
This type of relationship linking is common in Web APIs.

### Tasks
1. use the SWAPI films endpoint to return the data for all Star Wars films.
2. isolate/find the film data for the film title *A New Hope*
3. use the starships array from A New Hope to make HTTP requests for the data for all
 of the starships featured in this one film.
4. using the starships array attribute 'films', for each starship associated with *A New Hope*,
what is the name of the starship that has appeared (by count) in the greatest number of films.
You may ignore any ties.

### Task Logic
* use the films endpoint to return all films.
* for each returned film, find the film with title *A New Hope*
* For that film, save variable reference to the starships array
* For each starship endpoint for the film, request the starship endpoint
* For each returned starship save the count of items in the films list
* Find the max count of films
* Display the starship name and max count of films

**NOTE:** It is impossible to complete this task without reviewing (and understanding) the technical documents that describe the API.


In [9]:
import requests
import json
from requests.exceptions import HTTPError
from pprint import pprint

# The operator * is used here to pass along any additional arguments
# to .get() that many have been provided by the user. Ex: params, headers, timeout, etc.
# How does that work?  See https://realpython.com/python-kwargs-and-args/

def url_request(url:str, *args, **kwargs) -> requests.Response:
    try:
        response = requests.get(url, *args, **kwargs)
        # If the response was successful, no Exception will be raised
        # by the following line
        response.raise_for_status()

    except HTTPError as http_err:
        # Cannot proceed
        print(f'HTTP error occurred: {http_err}')
        raise
    except Exception as err:
        # Cannot proceed
        print(f'Other error occurred: {err}')
        raise
    else:
        print(f'Success: {response.status_code}')

    return response

def start_wars_main(url, target_film):
    response = url_request(url, timeout=10)
    json_as_py = response.json()
    #pprint(json_as_py, indent=2, compact=True)

    film_list = json_as_py['results']
    for film_dict in film_list:
        film_title = film_dict['title']
        if film_title == target_film:
            starships_list = film_dict['starships']
            
            # done here
            break
    else:
        raise ValueError(f'Target film {target_film} not found.')

    # for each starship in the target film, fetch the data for the starship endpoint
    # use a dict to keep track the ship names and the count of the movies
    # in which the ship was used
    #
    cnt_of_films_by_sship = {}
    for sship_epnt in starships_list:
    
        r = url_request(sship_epnt, timeout=5)
        ss_json_as_py = r.json()
        ss_name = ss_json_as_py['name']
        films_list = ss_json_as_py['films']
        cnt = len(films_list)
        cnt_of_films_by_sship[ss_name] = cnt

    max_cnt_key = max(cnt_of_films_by_sship, key=cnt_of_films_by_sship.get)

    ss_cnt = cnt_of_films_by_sship[max_cnt_key]
    ss_nm = max_cnt_key

    print(f"Starship '{ss_nm}' was used in {ss_cnt} films.")

    #################################################
    # Alternative solution that handles ties - there are ties!

    # Sort dict by its values in descending order
    # what is the max count
    max_cnt = max(cnt_of_films_by_sship.values())

    # print ships with max_cnt film appearances
    for k,v in cnt_of_films_by_sship.items():
        if v == max_cnt:
            print(f"Starship '{k}' was used in {v} films.")

if __name__ == '__main__':
    FILMS_URL = 'https://swapi.dev/api/films/'
    TARGET_FILM = 'A New Hope'
    start_wars_main(url=FILMS_URL, target_film=TARGET_FILM)

Success: 200
Success: 200
Success: 200
Success: 200
Success: 200
Success: 200
Success: 200
Success: 200
Success: 200
Starship 'CR90 corvette' was used in 3 films.
Starship 'CR90 corvette' was used in 3 films.
Starship 'Star Destroyer' was used in 3 films.
Starship 'Millennium Falcon' was used in 3 films.
Starship 'Y-wing' was used in 3 films.
Starship 'X-wing' was used in 3 films.
